In [ ]:
import json
import re
from tqdm import tqdm
import nltk
from gensim.models import Word2Vec
import os
import stanza

DATA_FILE_PATH = './billinfo-skl9.json' 
MODEL_FILE_PATH = './word2vec_rada_stanza.model'
SEED_KEYWORDS = {
    'Аграрна': ['земельний', 'субсидія', 'експортний', 'квота', 'фітосанітарний', 'ринок землі', 'сільськогосподарський', 'аграрний', 'фермерський', 'оренда землі'],
    'Соціальна': ['трудовий', 'соціальний', 'пенсійний', 'охорона праці', 'зайнятість', 'профспілка', 'внесок'],
    'Корпоративна': ['оподаткування', 'податок', 'валютний', 'корпоративний', 'акціонерний', 'цінні папери', 'злиття та поглинання', 'ПДВ']
}

try:
    print("Initializing Stanza pipeline for Ukrainian...")
    nlp = stanza.Pipeline('uk', processors='tokenize,lemma')
    print("Stanza pipeline initialized.")
except Exception as e:
    print(f"Failed to initialize Stanza. It might be downloading the model. Error: {e}")
    print("Downloading the Stanza model for Ukrainian ('uk')...")
    stanza.download('uk')
    nlp = stanza.Pipeline('uk', processors='tokenize,lemma')
    print("Stanza pipeline initialized after download.")


try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    print("Downloading NLTK stopwords...")
    nltk.download('stopwords')
ukrainian_stopwords = nltk.corpus.stopwords.words('russian')
custom_stopwords = ['проект', 'закон', 'україна', 'щодо', 'про', 'внесення', 'зміна', 'до', 'деякий', 'акт', 'кодекс']
ukrainian_stopwords.extend(custom_stopwords)


def preprocess_text_stanza(text, nlp_pipeline):
    """Очищує та лематизує текст за допомогою Stanza."""
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    doc = nlp_pipeline(text)
    lemmatized_tokens = [
        word.lemma 
        for sent in doc.sentences for word in sent.words
        if word.lemma not in ukrainian_stopwords and len(word.lemma) > 2
    ]
    return lemmatized_tokens


def train_or_load_model(processed_titles):
    """Тренує модель Word2Vec або завантажує, якщо вона вже існує."""
    if os.path.exists(MODEL_FILE_PATH):
        print(f"Loading existing model from {MODEL_FILE_PATH}...")
        model = Word2Vec.load(MODEL_FILE_PATH)
        return model
    
    print("Training Word2Vec model... (this may take a few minutes)")
    model = Word2Vec(sentences=processed_titles, vector_size=150, window=5, min_count=5, workers=4)
    model.save(MODEL_FILE_PATH)
    print(f"Model trained and saved to {MODEL_FILE_PATH}")
    return model

def find_related_keywords(model, seed_keywords, top_n=10):
    """Знаходить семантично близькі слова."""
    analysis_results = {}
    for category, keywords in seed_keywords.items():
        analysis_results[category] = {}
        doc = nlp(" ".join(keywords))
        lemmatized_keywords = {word.lemma for sent in doc.sentences for word in sent.words}
        
        for keyword in lemmatized_keywords:
            if keyword in model.wv:
                similar_words = model.wv.most_similar(keyword, topn=top_n)
                analysis_results[category][keyword] = similar_words
            else:
                analysis_results[category][keyword] = []
    return analysis_results


def main():
    print("Step 1: Loading and cleaning data...")
    try:
        with open(DATA_FILE_PATH, 'r', encoding='utf-8') as f:
            raw_text = f.read()
        
        cleaned_text = re.sub(r'[\x00-\x1F]', ' ', raw_text)
        
        data = json.loads(cleaned_text)

    except FileNotFoundError:
        print(f"ERROR: Data file not found at {DATA_FILE_PATH}")
        return
    except json.JSONDecodeError as e:
        print(f"ERROR: Failed to parse JSON even after cleaning. Error: {e}")
        return

    titles = [item['name'] for item in data if 'name' in item and item['name']]
    
    print("Step 2: Preprocessing titles with Stanza...")
    processed_titles = [preprocess_text_stanza(title, nlp) for title in tqdm(titles, desc="Preprocessing titles")]
    
    print("\nStep 3: Training or loading Word2Vec model...")
    model = train_or_load_model(processed_titles)
    
    print("\nStep 4: Analyzing keywords to find nearest neighbors...")
    analysis_results = find_related_keywords(model, SEED_KEYWORDS)
    
    print("\n--- ANALYSIS RESULTS ---")
    for category, keywords_data in analysis_results.items():
        print(f"\n--- Категорія: {category.upper()} ---")
        for seed_word, suggestions in keywords_data.items():
            if suggestions:
                suggested_list = [f"{word} ({score:.2f})" for word, score in suggestions]
                print(f"  Для '{seed_word}': {', '.join(suggested_list)}")
            else:
                print(f"  Для '{seed_word}': (не знайдено близьких слів у моделі)")

if __name__ == "__main__":
    main()


2025-09-30 22:50:29 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


Initializing Stanza pipeline for Ukrainian...


2025-09-30 22:50:29 WARNING: Language uk package default expects mwt, which has been added
2025-09-30 22:50:29 INFO: Loading these models for language: uk (Ukrainian):
| Processor | Package     |
---------------------------
| tokenize  | iu          |
| mwt       | iu          |
| lemma     | iu_nocharlm |

2025-09-30 22:50:29 INFO: Using device: cpu
2025-09-30 22:50:29 INFO: Loading: tokenize
2025-09-30 22:50:29 INFO: Loading: mwt
2025-09-30 22:50:29 INFO: Loading: lemma
2025-09-30 22:50:30 INFO: Done loading processors!


Stanza pipeline initialized.
Step 1: Loading and cleaning data...
Step 2: Preprocessing titles with Stanza...


Preprocessing titles: 100%|██████████| 13199/13199 [03:54<00:00, 56.21it/s]



Step 3: Training or loading Word2Vec model...
Training Word2Vec model... (this may take a few minutes)
Model trained and saved to ./word2vec_rada_stanza.model

Step 4: Analyzing keywords to find nearest neighbors...

--- ANALYSIS RESULTS ---

--- Категорія: АГРАРНА ---
  Для 'сільськогосподарський': фермерський (0.98), регіональний (0.98), майбутній (0.97), машинобудування (0.97), вугільна (0.97), оператор (0.97), ціль (0.97), пенсійна (0.97), напрям (0.97), інформаційна (0.97)
  Для 'оренда': приватизація (0.98), комунальний (0.96), передача (0.91), майна (0.89), майний (0.89), природнозаповідне (0.89), обєкт (0.87), державні (0.87), земля (0.87), вугледобувний (0.87)
  Для 'земельний': ділянка (0.97), земля (0.91), розбудова (0.85), продаж (0.84), інфраструктури (0.83), відведення (0.83), сільськогосподарського (0.81), комунальний (0.81), цифровати (0.80), передача (0.80)
  Для 'фермерський': машинобудування (0.98), сільськогосподарський (0.98), меліорація (0.98), гідротехнічний (0.